In [3]:
!pip install swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 93.4 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16505 sha256=527e694bb973d05802f55e1807a171509adb01bc45cd06ad9512e6ec322340ad
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 2.2.1
    Uninstalling cloudpickle-2.2.1:
      Successfully uninstalled cloudpickle-2.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
snowflake-snowpark-python 1.26.0 requires cloudpickle!=2.1.0,!=2.2.0,<=2.2.1,>=1.6.0, but you have cloudpickle 3.1.1 which is incompatible.

[notice] A new release of p

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import ast
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
import swifter
import re
import os

nltk.download('punkt')
nltk.download('vader_lexicon')

/root/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [3]:
dtype_dict = {
    'video_id': str,
    'publishedAt': str,
    'channel_id': str,
    'title': str,
    'description': str,
    'duration': 'float64',
    'view': 'float64',
    'like': 'float64',
    'comment': 'float64',
    'sponsored': 'bool',
    'collection_time': str,
    'description_channel': str
}
parse_dates=['publishedAt']
df = pd.read_csv("cleaned_data.csv", parse_dates=parse_dates, dtype=dtype_dict)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74301 entries, 0 to 74300
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   video_id             74301 non-null  object             
 1   publishedAt          74301 non-null  datetime64[ns, UTC]
 2   channel_id           74301 non-null  object             
 3   title                74301 non-null  object             
 4   description          74207 non-null  object             
 5   tags                 74301 non-null  object             
 6   duration             74301 non-null  float64            
 7   view                 74297 non-null  float64            
 8   like                 74240 non-null  float64            
 9   comment              74191 non-null  float64            
 10  sponsored            74301 non-null  bool               
 11  collection_time      74301 non-null  object             
 12  description_channe

,video_id,publishedAt,channel_id,title,description,tags,duration,view,like,comment,sponsored,collection_time,description_channel
0,2HjctXU6N5I,2021-06-10 14:13:24+00:00,UCFl1mGlf5j0Qno1Kxnyv6FA,Why Time is the Most Valuable Resource in Leag...,Why Time is the Most Valuable Resource in Leag...,"['league of legends', 'league of legends pro g...",12.616667,61531.0,1871.0,79.0,False,2025-02-17 23:52:24.376342+00:00,Skill Capped's League of Legends guides are th...
1,Yp-hUcvXnes,2021-09-29 10:59:50+00:00,UCFl1mGlf5j0Qno1Kxnyv6FA,3 BEST Champions to ESCAPE LOW ELO for END of ...,3 BEST CHAMPIONS to ESCAPE LOW ELO for EVERY R...,"['best champions', 'best champs end of season'...",17.350000,74328.0,1470.0,139.0,False,2025-02-17 23:52:24.376342+00:00,Skill Capped's League of Legends guides are th...
2,N5-AMeWwxqg,2019-09-06 14:30:02+00:00,UCFl1mGlf5j0Qno1Kxnyv6FA,How Does G2 Wunder Make Challengers Look BRONZ...,http://www.skill-capped.com/lol\nSubscribe: ht...,"['G2 esports', 'G2', 'G2 Wunder', 'Wunder Guid...",9.033333,178024.0,2909.0,171.0,False,2025-02-17 23:52:24.376342+00:00,Skill Capped's League of Legends guides are th...
3,-OzarB6x0q4,2021-12-18 14:00:01+00:00,UCFl1mGlf5j0Qno1Kxnyv6FA,How to Play Like a YASUO MAIN! - ULTIMATE YASU...,►Get the RANK You’ve Always Wanted: https://ww...,"['yasuo guide', 'yasuo', 'yasuo runes', 'yasuo...",10.500000,612349.0,14706.0,353.0,False,2025-02-17 23:52:24.376342+00:00,Skill Capped's League of Legends guides are th...
4,J6wmhw5m0EU,2022-07-21 13:00:12+00:00,UCFl1mGlf5j0Qno1Kxnyv6FA,How I Would Learn Mid Lane (If I Could Start O...,"►IMPROVE FAST, RANK UP, GUARANTEED: https://ww...","['league of legends', 'league of legends guide...",10.900000,307554.0,8251.0,242.0,False,2025-02-17 23:52:24.376342+00:00,Skill Capped's League of Legends guides are th...


In [4]:
df.loc[df['title'].isna(), 'title'] = ""
df.loc[df['description'].isna(), 'description'] = ""
df.loc[df['tags'].isna(), 'tags'] = "[]"

In [5]:
# generate likes-to-views ratio
# drop like and view cols
df['likes_to_views'] = df['like'] / df['view']
df = df.drop(columns=['like', 'view'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74301 entries, 0 to 74300
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   video_id             74301 non-null  object             
 1   publishedAt          74301 non-null  datetime64[ns, UTC]
 2   channel_id           74301 non-null  object             
 3   title                74301 non-null  object             
 4   description          74301 non-null  object             
 5   tags                 74301 non-null  object             
 6   duration             74301 non-null  float64            
 7   comment              74191 non-null  float64            
 8   sponsored            74301 non-null  bool               
 9   collection_time      74301 non-null  object             
 10  description_channel  69073 non-null  object             
 11  likes_to_views       74236 non-null  float64            
dtypes: bool(1), dateti

In [6]:
# converting tags to an easy to use list string
def converting_tags_to_string(text):
    text = ast.literal_eval(text)
    return "|||".join(text)
df['tags'] = df['tags'].apply(converting_tags_to_string)

# get the length for title, description, and tags 
# (since sentimental analysis wont be focusing at the length of the text)
df['title_len'] = df['title'].apply(len)
df['description_len'] = df['description'].apply(len)
df['num_tags'] = df['tags'].str.split('|||').apply(len)

In [13]:
# vader score
sia = SentimentIntensityAnalyzer()
def sentiment_analysis(text):
    if pd.isnull(text):
        return 0
    return sia.polarity_scores(text)['compound']

if not os.path.exists("vader.csv"):
    df['description_sentiment'] = df['description'].swifter.apply(sentiment_analysis)
    df['title_sentiment'] = df['title'].swifter.apply(sentiment_analysis)
    df.to_csv("vader.csv", index=False)
else:
    df = pd.read_csv("vader.csv")

In [15]:
# Average engegement over the last 20 videos for each channel
df = df.sort_values(by=['channel_id', 'publishedAt'], ascending=True)

df['avg_engagement_last_20'] = df.groupby('channel_id')['likes_to_views'].transform(
    lambda x: x.rolling(window=20, min_periods=1).mean()
)

In [17]:
# Filter out hashtags from description
def remove_hastags(text):
        return re.sub(r'#\w+', '', text)

df['description'] = df['description'].apply(remove_hastags)

def syllable_count(word):
    word = word.lower()
    count = 0
    vowels = "aeiou"
    prev_char_is_vowel = False
    
    for char in word:
        is_vowel = char in vowels
        if is_vowel and not prev_char_is_vowel:
            count += 1
        prev_char_is_vowel = is_vowel
    
    # Handle silent e at the end
    if word.endswith('e'):
        count -= 1
    # Ensure at least one syllable
    return max(1, count)

def flesch_reading_ease(text):
    sentences = sent_tokenize(text)
    words = word_tokenize(text)
    syllables = sum(syllable_count(word) for word in words)
    num_sentences = len(sentences)
    num_words = len(words)
    if num_sentences == 0 or num_words == 0:
        return 0
    return 206.835 - 1.015 * (num_words / num_sentences) - 84.6 * (syllables / num_words)

# Calculate FRE scores
if not os.path.exists('fk_scores.csv'):
    df['fre_title'] = df['title'].swifter.apply(flesch_reading_ease)
    df['fre_description'] = df['description'].swifter.apply(flesch_reading_ease)
    df.to_csv('fk_scores.csv', index=False)
else:
    df = pd.read_csv('fk_scores.csv')

df.head()

Pandas Apply: 100%|██████████| 74301/74301 [01:46<00:00, 695.57it/s]


,video_id,publishedAt,channel_id,title,description,tags,duration,comment,sponsored,collection_time,description_channel,likes_to_views,title_len,description_len,num_tags,avg_engagement_last_20,fre_title,fre_description
6771,MU3TgcrmPDU,2019-03-14 21:03:26+00:00,UC15FhQd0x8dYPkFsH0bTsMg,BLUEFACE PLAYS IMVU!,Created By AllStar@IMVU ★\nhttp://avatars.imvu...,IMVU|||blueface,1.516667,354.0,False,2025-02-17 23:52:24.376342+00:00,"God 1st 💛 Live, Laugh, Love! - I love my suppo...",0.026977,20,480,17,0.026977,75.875,-17.743134
6764,Wr0BsKzwHuo,2019-08-21 15:16:26+00:00,UC15FhQd0x8dYPkFsH0bTsMg,BLOOD ND CRIP GOES SHOPPING,Subscribe Thanks ★ Read Below ⬇️\nCreated By A...,IMVU|||GANG|||euina|||funny|||gameplay|||comme...,6.733333,64.0,False,2025-02-17 23:52:24.376342+00:00,"God 1st 💛 Live, Laugh, Love! - I love my suppo...",0.046337,27,489,68,0.036657,100.240,-26.163571
6624,EY3dVAeb9zA,2019-08-24 20:23:16+00:00,UC15FhQd0x8dYPkFsH0bTsMg,TEEJAYX6 SCAMS IMVU,Subscribe if you love your mom 💛 Read Below 👇\...,IMVU|||TEEJAYx6|||funny|||scam|||trolling,2.600000,61.0,False,2025-02-17 23:52:24.376342+00:00,"God 1st 💛 Live, Laugh, Love! - I love my suppo...",0.049857,19,421,43,0.041057,62.790,-7.878261
6569,2Yjz9vR6iGM,2019-08-25 15:44:04+00:00,UC15FhQd0x8dYPkFsH0bTsMg,HOW TO GET A GIRLFRIEND,Subscribe if you love your mom 💛 Read Below 👇\...,IMVU|||BOYFRIEND|||GIRLFRIEND|||HOW TO GET A G...,16.833333,161.0,False,2025-02-17 23:52:24.376342+00:00,"God 1st 💛 Live, Laugh, Love! - I love my suppo...",0.037240,23,730,126,0.040103,100.240,7.155728
6742,dvdPNhSOkvk,2019-08-26 16:41:48+00:00,UC15FhQd0x8dYPkFsH0bTsMg,BLOODS VS CRIPS TURF WAR,Subscribe if you love your mom 💛 Read Below 👇\...,IMVU|||funny|||trolling|||blood|||crips,13.933333,190.0,False,2025-02-17 23:52:24.376342+00:00,"God 1st 💛 Live, Laugh, Love! - I love my suppo...",0.037206,24,654,41,0.039523,117.160,12.343837


In [60]:
# Finding the rows with minimum and maximum values for fk_title and fk_description
min_fre_title_row = df[df['fre_title'] == df['fre_title'].min()]
max_fre_title_row = df[df['fre_title'] == df['fre_title'].max()]

min_fre_description_row = df[df['fre_description'] == df['fre_description'].min()]
max_fre_description_row = df[df['fre_description'] == df['fre_description'].max()]

print("Row with minimum fre_title:")
print(min_fre_title_row['title'].values[0])
print("\nRow with maximum fre_title:")
print(max_fre_title_row['title'].values[0])

print("\nRow with minimum fre_description:")
print(min_fre_description_row['description'].values[0])
print("\nRow with maximum fre_description:")
print(max_fre_description_row['description'].values[0])

Row with minimum fk_title:
HAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHA

Row with maximum fk_title:
drugs

Row with minimum fk_description:
I Built This Squad in 2 Months 🤩🔥 | 124 OVR Squad in 2 months - FIFA mobile 

#fifamobile #fifa23 #fifa22 #ecshani 

Music by YT library 🎶 
Editing app Capcut 


Ignore tags :

Utoty pack opening 
#utoty
#utotypackopening
#fifamobile22 
#fifamobilepackopening
#Ronaldo
#fifamobile
#FIFAMOBILE


FIFA MOBILE 22 NEWS
FIFA MOBILE 22 UPDATES
FIFA MOBILE 22 NEW ICONS
FIFA MOBILE 22 PRIME ICON 
FIFA MOBILE 21 NEW EVENT

FIFA MOBILE 22 PACK OPENING 
FIFA MOBILE 22 TEAM UPGRADE 
FIFA MOBILE 22 VIDEOS 
FIFA MOBILE 22 BEST PACK OPENING 
FIFA MOBILE 22 TOTW
FIFA MOBILE 22 GAMEPLAY 
FIFA MOBILE 22 LEGENDS
FIFA MOBILE 22 GUIDE
HOW TO PLAY FIFA MOBILE 22
FIFA MOBILE 22 DIVISION RIVALS
FIFA MOBILE 22 EMOTES
FIFA MOBILE 22 NEW FEATURE 
FIFA MOBILE 22 NEW UPDATES 
HOW TO PLAY FIFA MOBILE 22

FIFA MOBILE 22 EVERY

In [19]:
# hours since last upload

#Sorting by channel and time
df = df.sort_values(['channel_id', 'publishedAt'])

#Computing time difference per channel
df['hours_since_last_upload'] = df.groupby('channel_id')['publishedAt'].diff().dt.total_seconds() / 3600
df.info()
df.head(5000)


<class 'pandas.core.frame.DataFrame'>
Index: 74301 entries, 6771 to 35791
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   video_id                 74301 non-null  object             
 1   publishedAt              74301 non-null  datetime64[ns, UTC]
 2   channel_id               74301 non-null  object             
 3   title                    74301 non-null  object             
 4   description              74301 non-null  object             
 5   tags                     74301 non-null  object             
 6   duration                 74301 non-null  float64            
 7   comment                  74191 non-null  float64            
 8   sponsored                74301 non-null  bool               
 9   collection_time          74301 non-null  object             
 10  description_channel      69073 non-null  object             
 11  likes_to_views           74236

,video_id,publishedAt,channel_id,title,description,tags,duration,comment,sponsored,collection_time,description_channel,likes_to_views,title_len,description_len,num_tags,avg_engagement_last_20,fre_title,fre_description,hours_since_last_upload
6771,MU3TgcrmPDU,2019-03-14 21:03:26+00:00,UC15FhQd0x8dYPkFsH0bTsMg,BLUEFACE PLAYS IMVU!,Created By AllStar@IMVU ★\nhttp://avatars.imvu...,IMVU|||blueface,1.516667,354.0,False,2025-02-17 23:52:24.376342+00:00,"God 1st 💛 Live, Laugh, Love! - I love my suppo...",0.026977,20,480,17,0.026977,75.875000,-17.743134,NaN
6764,Wr0BsKzwHuo,2019-08-21 15:16:26+00:00,UC15FhQd0x8dYPkFsH0bTsMg,BLOOD ND CRIP GOES SHOPPING,Subscribe Thanks ★ Read Below ⬇️\nCreated By A...,IMVU|||GANG|||euina|||funny|||gameplay|||comme...,6.733333,64.0,False,2025-02-17 23:52:24.376342+00:00,"God 1st 💛 Live, Laugh, Love! - I love my suppo...",0.046337,27,489,68,0.036657,100.240000,-26.163571,3834.216667
6624,EY3dVAeb9zA,2019-08-24 20:23:16+00:00,UC15FhQd0x8dYPkFsH0bTsMg,TEEJAYX6 SCAMS IMVU,Subscribe if you love your mom 💛 Read Below 👇\...,IMVU|||TEEJAYx6|||funny|||scam|||trolling,2.600000,61.0,False,2025-02-17 23:52:24.376342+00:00,"God 1st 💛 Live, Laugh, Love! - I love my suppo...",0.049857,19,421,43,0.041057,62.790000,-7.878261,77.113889
6569,2Yjz9vR6iGM,2019-08-25 15:44:04+00:00,UC15FhQd0x8dYPkFsH0bTsMg,HOW TO GET A GIRLFRIEND,Subscribe if you love your mom 💛 Read Below 👇\...,IMVU|||BOYFRIEND|||GIRLFRIEND|||HOW TO GET A G...,16.833333,161.0,False,2025-02-17 23:52:24.376342+00:00,"God 1st 💛 Live, Laugh, Love! - I love my suppo...",0.037240,23,730,126,0.040103,100.240000,7.155728,19.346667
6742,dvdPNhSOkvk,2019-08-26 16:41:48+00:00,UC15FhQd0x8dYPkFsH0bTsMg,BLOODS VS CRIPS TURF WAR,Subscribe if you love your mom 💛 Read Below 👇\...,IMVU|||funny|||trolling|||blood|||crips,13.933333,190.0,False,2025-02-17 23:52:24.376342+00:00,"God 1st 💛 Live, Laugh, Love! - I love my suppo...",0.037206,24,654,41,0.039523,117.160000,12.343837,24.962222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33750,hTjS5HrQN0Q,2015-12-09 22:30:00+00:00,UC7U3eB5rsAGWPrR97unKRFQ,How To - First Kiss (Minecraft Roleplay),Previous Video: https://www.youtube.com/watch?...,Minecraft (Award-Winning Work)|||minecraft how...,6.283333,689.0,False,2025-02-18 00:07:45.237340+00:00,NaN,0.009736,40,297,391,0.015982,75.500000,1.859767,24.500000
34760,Z1tDyNsEbNw,2015-12-10 22:30:00+00:00,UC7U3eB5rsAGWPrR97unKRFQ,Lying Girlfriend | Minecraft School [S3: Ep.2 ...,Bri is a liar?!\nPrevious Video: https://www.y...,Minecraft (Award-Winning Work)|||minecraft sch...,6.500000,1674.0,False,2025-02-18 00:07:45.237340+00:00,NaN,0.012902,75,313,387,0.016040,50.470769,30.032806,24.000000
33888,qPpGsTrfaBE,2015-12-11 23:00:00+00:00,UC7U3eB5rsAGWPrR97unKRFQ,Date Night | Yandere University [S2: Ep.1 Mine...,We need to go on a date with the crazy Zoe!\nP...,Minecraft (Award-Winning Work)|||minecraft yan...,7.683333,2153.0,False,2025-02-18 00:07:45.237340+00:00,NaN,0.013137,71,341,435,0.015937,43.963077,34.333409,24.500000
33558,s1zJhsNfVCo,2015-12-12 22:30:01+00:00,UC7U3eB5rsAGWPrR97unKRFQ,Out of Time | Minecraft Life [S4: Ep.2 Minecra...,We're running out of time!\nPrevious Video: ht...,Minecraft (Award-Winning Work)|||minecraft lif...,11.366667,485.0,False,2025-02-18 00:07:45.237340+00:00,NaN,0.014542,68,324,322,0.016001,59.682143,29.180000,23.500278


In [21]:
# standard dev between video uploads
time_diff_std = df.groupby('channel_id')['hours_since_last_upload'].std()
df['std_time_diff'] = df['channel_id'].map(time_diff_std)

# Comment Handling

In [23]:
comment_dtype_dict = {
    'video_id': str,
    'like_count': 'float64',
    'text_original': str,
    'published_at': str
}
usecols = ['video_id', 'like_count', 'text_original', 'published_at']
parse_dates = ['published_at']

df_comment = pd.read_csv("gaming_comments.csv", usecols=usecols, dtype=comment_dtype_dict, parse_dates=parse_dates)
df_comment.info()
df_comment.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920105 entries, 0 to 920104
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype              
---  ------         --------------   -----              
 0   video_id       920105 non-null  object             
 1   text_original  919838 non-null  object             
 2   like_count     920105 non-null  float64            
 3   published_at   920105 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(1), object(2)
memory usage: 28.1+ MB


,video_id,text_original,like_count,published_at
0,2HjctXU6N5I,"As Ekko main, sometimes I wish I could R back ...",317.0,2021-06-10 14:52:01+00:00
1,2HjctXU6N5I,"If we really value time, we wouldn't be sittin...",187.0,2021-06-10 14:44:15+00:00
2,2HjctXU6N5I,That's why my mains are zilean and ekko,62.0,2021-06-10 14:19:40+00:00
3,2HjctXU6N5I,"I found it really interesting, this makes me m...",10.0,2021-06-10 14:48:00+00:00
4,2HjctXU6N5I,Very nice concept to give us. I like the anima...,14.0,2021-06-10 14:25:08+00:00


In [25]:
average_like_count = df_comment.groupby('video_id')['like_count'].mean().reset_index()
average_like_count.rename(columns={'like_count': 'avg_like_count'}, inplace=True)

In [27]:
## Set blank missing comments to empty strings (if any exist)
df_comment.loc[df_comment['text_original'].isna()] = "" 

In [29]:
## Average comment length
df_comment['comment_len'] = df_comment['text_original'].apply(len)
average_comment_len = df_comment.groupby('video_id')['comment_len'].mean().reset_index()

In [31]:
## Average Flesch-Kincaid score
df_comment['fre_comment'] = df_comment['text_original'].swifter.apply(flesch_reading_ease)
average_comment_fre = df_comment.groupby('video_id')['fre_comment'].mean().reset_index()

Pandas Apply: 100%|██████████| 920105/920105 [04:07<00:00, 3711.36it/s]


In [32]:
## Average Vader score (might need preprocess depending on Kaiden implementation)
df_comment['vader_comment'] = df_comment['text_original'].swifter.apply(sentiment_analysis)
average_comment_vader = df_comment.groupby('video_id')['vader_comment'].mean().reset_index()

Pandas Apply: 100%|██████████| 920105/920105 [03:17<00:00, 4664.43it/s]


In [35]:
## Combine average comment statistic dfs into one df
avg_comment_df = pd.concat([
    average_like_count,
    average_comment_len['comment_len'],
    average_comment_fre['fre_comment'], 
    average_comment_vader['vader_comment']
], axis = 1)
avg_comment_df.to_csv('avg_comment_df.csv', index=False)

In [41]:
## Join the average values w original df
df = df.merge(avg_comment_df, on='video_id', how='inner')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70421 entries, 0 to 70420
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   video_id                 70421 non-null  object             
 1   publishedAt              70421 non-null  datetime64[ns, UTC]
 2   channel_id               70421 non-null  object             
 3   title                    70421 non-null  object             
 4   description              70421 non-null  object             
 5   tags                     70421 non-null  object             
 6   duration                 70421 non-null  float64            
 7   comment                  70421 non-null  float64            
 8   sponsored                70421 non-null  bool               
 9   collection_time          70421 non-null  object             
 10  description_channel      65193 non-null  object             
 11  likes_to_views           703

In [39]:
df.to_csv('derived_attributes_data.csv', index=False)

In [6]:
df = pd.read_csv("derived_attributes_data.csv")

In [60]:
df.head()

,video_id,publishedAt,channel_id,title,description,tags,duration,comment,sponsored,collection_time,...,fre_title,fre_description,hours_since_last_upload,std_time_diff,like_count,comment_len,fre_comment,vader_comment,description_sentiment_y,title_sentiment_y
0,MU3TgcrmPDU,2019-03-14 21:03:26+00:00,UC15FhQd0x8dYPkFsH0bTsMg,BLUEFACE PLAYS IMVU!,Created By AllStar@IMVU ★\nhttp://avatars.imvu...,0.0,1.516667,354.0,False,2025-02-17 23:52:24.376342+00:00,...,75.875,-17.743134,NaN,336.245364,230.800000,16.5,87.972500,0.73645,0.8286,0.3164
1,Wr0BsKzwHuo,2019-08-21 15:16:26+00:00,UC15FhQd0x8dYPkFsH0bTsMg,BLOOD ND CRIP GOES SHOPPING,Subscribe Thanks ★ Read Below ⬇️\nCreated By A...,0.0,6.733333,64.0,False,2025-02-17 23:52:24.376342+00:00,...,100.240,-26.163571,3834.216667,336.245364,13.500000,98.2,85.416921,0.29819,0.7650,0.0000
2,EY3dVAeb9zA,2019-08-24 20:23:16+00:00,UC15FhQd0x8dYPkFsH0bTsMg,TEEJAYX6 SCAMS IMVU,Subscribe if you love your mom 💛 Read Below 👇\...,0.0,2.600000,61.0,False,2025-02-17 23:52:24.376342+00:00,...,62.790,-7.878261,77.113889,336.245364,7.052632,5.0,121.220000,0.00000,0.8360,-0.5859
3,2Yjz9vR6iGM,2019-08-25 15:44:04+00:00,UC15FhQd0x8dYPkFsH0bTsMg,HOW TO GET A GIRLFRIEND,Subscribe if you love your mom 💛 Read Below 👇\...,0.0,16.833333,161.0,False,2025-02-17 23:52:24.376342+00:00,...,100.240,7.155728,19.346667,336.245364,78.500000,51.7,96.557984,0.51463,-0.4926,0.0000
4,dvdPNhSOkvk,2019-08-26 16:41:48+00:00,UC15FhQd0x8dYPkFsH0bTsMg,BLOODS VS CRIPS TURF WAR,Subscribe if you love your mom 💛 Read Below 👇\...,0.0,13.933333,190.0,False,2025-02-17 23:52:24.376342+00:00,...,117.160,12.343837,24.962222,336.245364,39.947368,45.7,88.223008,0.23751,0.7263,-0.5994


In [8]:
def z_score_normalize(col):
    mean = col.mean()
    std = col.std()
    n_col = (col - mean) / std
    return n_col

def log_normalize(col):
    n_col = np.log(col)
    return n_col

df['duration'] = log_normalize(df['duration'])
df['comment'] = z_score_normalize(df['comment']) # Mean: 1516.3399128100993 # std: 6154.523553093705
df['title_len'] = log_normalize(df['title_len'])
df['description_len'] = z_score_normalize(df['description_len']) # Mean: 874.9608071455958 # std: 653.4494568378558
df['fre_title'] = z_score_normalize(df['fre_title']) # Mean: 75.76084974375163 # std: 26.920986415279344
df['fre_description'] = z_score_normalize(df['fre_description']) # Mean: 39.18588579143162 # std: 34.987706879214166
df['fre_comment'] = z_score_normalize(df['fre_comment']) # Mean: 86.12505394659725 # std: 32.29057824184089
df['hours_since_last_upload'] = z_score_normalize(df['hours_since_last_upload']) # Mean: 42.49308977109649 # std: 239.78691629702098
df['comment_len'] = log_normalize(df['comment_len'])
df['fre_comment'] = z_score_normalize(df['fre_comment'])
df['avg_like_count'] = z_score_normalize(df['avg_like_count']) # Mean: 287.206441313211 std: 1184.0061151266134
df.head()

/root/venv/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,video_id,publishedAt,channel_id,title,description,tags,duration,comment,sponsored,collection_time,...,num_tags,avg_engagement_last_20,fre_title,fre_description,hours_since_last_upload,std_time_diff,avg_like_count,comment_len,fre_comment,vader_comment
0,MU3TgcrmPDU,2019-03-14 21:03:26+00:00,UC15FhQd0x8dYPkFsH0bTsMg,BLUEFACE PLAYS IMVU!,Created By AllStar@IMVU ★\nhttp://avatars.imvu...,IMVU|||blueface,0.416515,-0.188859,False,2025-02-17 23:52:24.376342+00:00,...,17,0.026977,0.004240,-1.627115,NaN,336.245364,-0.047640,2.803360,0.057213,0.73645
1,Wr0BsKzwHuo,2019-08-21 15:16:26+00:00,UC15FhQd0x8dYPkFsH0bTsMg,BLOOD ND CRIP GOES SHOPPING,Subscribe Thanks ★ Read Below ⬇️\nCreated By A...,IMVU|||GANG|||euina|||funny|||gameplay|||comme...,1.907070,-0.235979,False,2025-02-17 23:52:24.376342+00:00,...,68,0.036657,0.909296,-1.867783,15.812888,336.245364,-0.231170,4.587006,-0.021930,0.29819
2,EY3dVAeb9zA,2019-08-24 20:23:16+00:00,UC15FhQd0x8dYPkFsH0bTsMg,TEEJAYX6 SCAMS IMVU,Subscribe if you love your mom 💛 Read Below 👇\...,IMVU|||TEEJAYx6|||funny|||scam|||trolling,0.955511,-0.236467,False,2025-02-17 23:52:24.376342+00:00,...,43,0.041057,-0.481812,-1.345162,0.144382,336.245364,-0.236615,1.609438,1.086848,0.00000
3,2Yjz9vR6iGM,2019-08-25 15:44:04+00:00,UC15FhQd0x8dYPkFsH0bTsMg,HOW TO GET A GIRLFRIEND,Subscribe if you love your mom 💛 Read Below 👇\...,IMVU|||BOYFRIEND|||GIRLFRIEND|||HOW TO GET A G...,2.823361,-0.220218,False,2025-02-17 23:52:24.376342+00:00,...,126,0.040103,0.909296,-0.915469,-0.096529,336.245364,-0.176271,3.945458,0.323095,0.51463
4,dvdPNhSOkvk,2019-08-26 16:41:48+00:00,UC15FhQd0x8dYPkFsH0bTsMg,BLOODS VS CRIPS TURF WAR,Subscribe if you love your mom 💛 Read Below 👇\...,IMVU|||funny|||trolling|||blood|||crips,2.634284,-0.215507,False,2025-02-17 23:52:24.376342+00:00,...,41,0.039523,1.537802,-0.767185,-0.073110,336.245364,-0.208833,3.822098,0.064971,0.23751


# Dropping Columns used in the Derived Attributes

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70421 entries, 0 to 70420
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   video_id                 70421 non-null  object             
 1   publishedAt              70421 non-null  datetime64[ns, UTC]
 2   channel_id               70421 non-null  object             
 3   title                    70421 non-null  object             
 4   description              70421 non-null  object             
 5   tags                     70421 non-null  float64            
 6   duration                 70421 non-null  float64            
 7   comment                  70421 non-null  float64            
 8   sponsored                70421 non-null  bool               
 9   collection_time          70421 non-null  object             
 10  description_channel      65193 non-null  object             
 11  likes_to_views           703

In [10]:
numeric_df = df.drop(columns=['publishedAt', 'channel_id', 'title', 'description', 'tags',
            'collection_time', 'description_channel', 'num_tags', 'std_time_diff'])

numeric_df.to_csv('numeric.csv', index=False)

In [104]:
len(numeric_df.columns)

18

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=395a814d-9932-4e75-96c1-b9471a984f5a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>